this is folk of the1owls's "Ridge & Huber - 3 Pointer (M)"  with Japanese caption.  
https://www.kaggle.com/the1owl/ridge-huber-3-pointer-m  
just for my understanding

In [3]:
# ライブラリを読み込む
import pandas as pd
import numpy as np
from sklearn import *
import os, glob

/home/akiyama/anaconda3/envs/keras2/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/akiyama/anaconda3/envs/keras2/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/akiyama/anaconda3/envs/keras2/lib/python3.5/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will

In [4]:
# データを読み込む
datafiles = sorted(glob.glob('../input/**.csv')) # 全データのパスを取得
# 全データをpd.Dataframeとして読み込んでdictionaryにまとめる
datafiles = {os.path.basename(file)[:-4]: pd.read_csv(file, encoding='latin-1') for file in datafiles}

In [5]:
# 全データをリストアップ
for i, key in enumerate(datafiles):
    print(i, key)

0 feature1
1 MasseyOrdinals
2 TeamCoaches
3 RegularSeasonCompactResults
4 NCAATourneySeedRoundSlots
5 Cities
6 SecondaryTourneyCompactResults
7 NCAATourneySeeds
8 TeamSpellings
9 NCAATourneySlots
10 Seasons
11 NCAATourneyDetailedResults
12 SecondaryTourneyTeams
13 GameCities
14 Teams
15 Conferences
16 ConferenceTourneyGames
17 TeamConferences
18 SampleSubmissionStage1
19 NCAATourneyCompactResults
20 RegularSeasonDetailedResults


In [6]:
"""
SecondaryTourneyというカラムを追加して 'NCAA"もしくは'Regular'の値を与える
SecondaryTourneyはNCAAトーナメント以外のトーナメントのことでNIT, CBI, CIT, V16の４つがある．
ここにNCAA, Regularを追加して試合がどの大会に属するものかを表すカラムとして扱う．
"""
datafiles['NCAATourneyCompactResults']['SecondaryTourney'] = 'NCAA'
datafiles['NCAATourneyDetailedResults']['SecondaryTourney'] = 'NCAA'
datafiles['RegularSeasonCompactResults']['SecondaryTourney'] = 'Regular'
datafiles['RegularSeasonDetailedResults']['SecondaryTourney'] = 'Regular'

In [7]:
### Presets
# カテゴリ型のカラムを数値に置き換える準備
WLoc = {'A': 1, 'H': 2, 'N': 3}
SecondaryTourney = {'NIT': 1, 'CBI': 2, 'CIT': 3, 'V16': 4, 'Regular': 5 ,'NCAA': 6}

In [8]:
# NCAAトーナメントとレギュラーリーグの試合情報を連結

# コンパクト版
# NCAA, レギュラーシーズンの試合情報を連結
games = pd.concat((datafiles['NCAATourneyCompactResults'],datafiles['RegularSeasonCompactResults']), axis=0, ignore_index=True)
# さらにSecondaryTourney () の試合情報を連結
games = pd.concat((games,datafiles['SecondaryTourneyCompactResults']), axis=0, ignore_index=True)

# 詳細版
#games = pd.concat((datafiles['NCAATourneyDetailedResults'],datafiles['RegularSeasonDetailedResults']), axis=0, ignore_index=True)

games.reset_index(drop=True, inplace=True) # indexをリセット
# 試合会場情報をH(ome), A(way), N(eutral)から1, 2, 3 に変換
games['WLoc'] = games['WLoc'].map(WLoc)
# 大会形式情報をNIT, CBI, CIT, V16, Regular, NCAAから1, 2, 3, 4, 5, 6 に変換
games['SecondaryTourney'] = games['SecondaryTourney'].map(SecondaryTourney)
print("games.shape", games.shape)
games.head()

games.shape (154285, 9)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney
0,1985,136,1116,63,1234,54,3,0,6
1,1985,136,1120,59,1345,58,3,0,6
2,1985,136,1207,68,1250,43,3,0,6
3,1985,136,1229,58,1425,55,3,0,6
4,1985,136,1242,49,1325,38,3,0,6


In [9]:
###Add Ids
# カラム ID, IDTeam, Team1, Team2, IDTeam1, IDTeam2 を追加
# Team1, Team2は勝利チームと敗北チームをソートしたものなので入れ替わっている場合有り
games['ID'] = games.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['IDTeams'] = games.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['Team1'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[0], axis=1)
games['Team2'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[1], axis=1)
games['IDTeam1'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
games['IDTeam2'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
print("games.shape", games.shape)
games.head()

games.shape (154285, 15)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney,ID,IDTeams,Team1,Team2,IDTeam1,IDTeam2
0,1985,136,1116,63,1234,54,3,0,6,1985_1116_1234,1116_1234,1116,1234,1985_1116,1985_1234
1,1985,136,1120,59,1345,58,3,0,6,1985_1120_1345,1120_1345,1120,1345,1985_1120,1985_1345
2,1985,136,1207,68,1250,43,3,0,6,1985_1207_1250,1207_1250,1207,1250,1985_1207,1985_1250
3,1985,136,1229,58,1425,55,3,0,6,1985_1229_1425,1229_1425,1229,1425,1985_1229,1985_1425
4,1985,136,1242,49,1325,38,3,0,6,1985_1242_1325,1242_1325,1242,1325,1985_1242,1985_1325


In [10]:
###Add Seeds
# シード情報を読み込みIDTeamsとひも付けた形に変換
seeds = {'_'.join(map(str,[int(k1),k2])):int(v[1:3]) for k1, v, k2 in datafiles['NCAATourneySeeds'].values}
for key in sorted(seeds.keys())[:10]:
    print(key, seeds[key])

1985_1104 7
1985_1112 10
1985_1116 9
1985_1120 11
1985_1130 11
1985_1173 9
1985_1177 10
1985_1181 3
1985_1192 16
1985_1207 1


In [11]:
### Add 2018
# 2018年のシード順位を追加する．2018年のシード順位は2017年と同じとする．
if 2018 not in datafiles['NCAATourneySeeds']['Season'].unique():
    seeds = {**seeds, **{k.replace('2017_','2018_'):seeds[k] for k in seeds if '2017_' in k}}
print("2017年のDuke大学のシード順位:", seeds['2017_1181'], "2018年のDuke大学のシード順位:", seeds['2018_1181'])

2017年のDuke大学のシード順位: 2 2018年のDuke大学のシード順位: 2


In [12]:
# シード順位情報のカラムを追加. 情報がないチームは0を代入
games['Team1Seed'] = games['IDTeam1'].map(seeds).fillna(0)
games['Team2Seed'] = games['IDTeam2'].map(seeds).fillna(0)
games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney,ID,IDTeams,Team1,Team2,IDTeam1,IDTeam2,Team1Seed,Team2Seed
0,1985,136,1116,63,1234,54,3,0,6,1985_1116_1234,1116_1234,1116,1234,1985_1116,1985_1234,9.0,8.0
1,1985,136,1120,59,1345,58,3,0,6,1985_1120_1345,1120_1345,1120,1345,1985_1120,1985_1345,11.0,6.0
2,1985,136,1207,68,1250,43,3,0,6,1985_1207_1250,1207_1250,1207,1250,1985_1207,1985_1250,1.0,16.0
3,1985,136,1229,58,1425,55,3,0,6,1985_1229_1425,1229_1425,1229,1425,1985_1229,1985_1425,9.0,8.0
4,1985,136,1242,49,1325,38,3,0,6,1985_1242_1325,1242_1325,1242,1325,1985_1242,1985_1325,3.0,14.0


In [13]:
### Additional Features & Clean Up
# 特徴量を追加する
games['ScoreDiff'] = games['WScore'] - games['LScore'] # 点差
# 試合結果 (チーム1 = 勝利チーム なら 1 (勝ち))
games['Pred'] = games.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1)
# 点差の絶対値 (負け試合だと点差がマイナスになっているので)
games['ScoreDiffNorm'] = games.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
games['SeedDiff'] = games['Team1Seed'] - games['Team2Seed'] # シード順位差
games = games.fillna(-1) # NAを-1で埋める
games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney,ID,...,Team1,Team2,IDTeam1,IDTeam2,Team1Seed,Team2Seed,ScoreDiff,Pred,ScoreDiffNorm,SeedDiff
0,1985,136,1116,63,1234,54,3,0,6,1985_1116_1234,...,1116,1234,1985_1116,1985_1234,9.0,8.0,9,1.0,9,1.0
1,1985,136,1120,59,1345,58,3,0,6,1985_1120_1345,...,1120,1345,1985_1120,1985_1345,11.0,6.0,1,1.0,1,5.0
2,1985,136,1207,68,1250,43,3,0,6,1985_1207_1250,...,1207,1250,1985_1207,1985_1250,1.0,16.0,25,1.0,25,-15.0
3,1985,136,1229,58,1425,55,3,0,6,1985_1229_1425,...,1229,1425,1985_1229,1985_1425,9.0,8.0,3,1.0,3,1.0
4,1985,136,1242,49,1325,38,3,0,6,1985_1242_1325,...,1242,1325,1985_1242,1985_1325,3.0,14.0,11,1.0,11,-11.0


In [14]:
### Test Set
#　テストデータ作成
sub = datafiles['SampleSubmissionStage1']
sub['WLoc'] = 3 #N NCAAの試合会場は必ず中立
sub['SecondaryTourney'] = 6 #NCAA
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['Season'].astype(int)
sub['Team1'] = sub['ID'].map(lambda x: x.split('_')[1])
sub['Team2'] = sub['ID'].map(lambda x: x.split('_')[2])
sub['IDTeams'] = sub.apply(lambda r: '_'.join(map(str, [r['Team1'], r['Team2']])), axis=1)
sub['IDTeam1'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
sub['IDTeam2'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
sub['Team1Seed'] = sub['IDTeam1'].map(seeds).fillna(0)
sub['Team2Seed'] = sub['IDTeam2'].map(seeds).fillna(0)
sub['SeedDiff'] = sub['Team1Seed'] - sub['Team2Seed'] 
sub.head()

,ID,Pred,WLoc,SecondaryTourney,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,Team2Seed,SeedDiff
0,2014_1107_1110,0.5,3,6,2014,1107,1110,1107_1110,2014_1107,2014_1110,16,15,1
1,2014_1107_1112,0.5,3,6,2014,1107,1112,1107_1112,2014_1107,2014_1112,16,1,15
2,2014_1107_1113,0.5,3,6,2014,1107,1113,1107_1113,2014_1107,2014_1113,16,10,6
3,2014_1107_1124,0.5,3,6,2014,1107,1124,1107_1124,2014_1107,2014_1124,16,6,10
4,2014_1107_1140,0.5,3,6,2014,1107,1140,1107_1140,2014_1107,2014_1140,16,10,6


In [15]:
sub.tail()

,ID,Pred,WLoc,SecondaryTourney,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,Team2Seed,SeedDiff
9107,2017_1455_1458,0.5,3,6,2017,1455,1458,1455_1458,2017_1455,2017_1458,10,8,2
9108,2017_1455_1462,0.5,3,6,2017,1455,1462,1455_1462,2017_1455,2017_1462,10,11,-1
9109,2017_1457_1458,0.5,3,6,2017,1457,1458,1457_1458,2017_1457,2017_1458,13,8,5
9110,2017_1457_1462,0.5,3,6,2017,1457,1462,1457_1462,2017_1457,2017_1462,13,11,2
9111,2017_1458_1462,0.5,3,6,2017,1458,1462,1458_1462,2017_1458,2017_1462,8,11,-3


In [18]:
# 説明変数を選択
col = [c for c in x1.columns if c not in [
    'ID', 
    'Team1',
    'Team2', 
    'IDTeams',
    'IDTeam1',
    'IDTeam2',
    'Pred',
    'DayNum', 
    'WTeamID', 
    'WScore', 
    'LTeamID', 
    'LScore', 
    'NumOT', 
    'ScoreDiff']]
col

['Season',
 'WLoc',
 'SecondaryTourney',
 'Team1Seed',
 'Team2Seed',
 'ScoreDiffNorm',
 'SeedDiff',
 'Inter']

In [95]:
sorted(x1['Inter'].unique())

In [96]:
"""
HuberRegressorモデルをトレーニング
HuberRegressorはoutlierにつよいregressorとのこと
"""

reg = linear_model.HuberRegressor()
reg.fit(x1[col], x1['Pred'])
pred = reg.predict(x2[col]).clip(0.05, 0.95)
print('Log Loss:', metrics.log_loss(x2['Pred'], pred))
test['Pred'] = reg.predict(test[col])

In [30]:
### Add Validation
"""
difference between C and sumbission deribe from ScoreDiffNorm
local CV in original use reald ScoreDiff and 
in submission estimated ScoreDiff



"""
results = []
for season in sub['Season'].unique(): # 年度ごとに処理
    print(season)
    # 前年度以前のNCAAのデータを抜き出す
    x1 = games[((games['Season']<int(season)) & (games['SecondaryTourney']==6))]
    # 同年度以前のNCAA以外のデータを抜き出し追加
    x1 = pd.concat((x1,games[((games['Season']<int(int(season)+1)) & (games['SecondaryTourney']!=6))]), axis=0, 
                   ignore_index=True)
    # 同年度のNCAAのデータを抜き出す
    x2 = games[((games['Season']==int(season)) & (games['SecondaryTourney']==6))]
    test = sub[sub['Season']==season]

    sdn = x1.groupby(['IDTeams'], as_index=False)[['ScoreDiffNorm']].mean() # 同じ対戦カードの過去の点差の平均
    test = pd.merge(test, sdn, how='left', on=['IDTeams'])
    test['ScoreDiffNorm'] = test['ScoreDiffNorm'].fillna(0.)
    
    x2 = x2.drop("ScoreDiffNorm", axis=1)
    x2 = pd.merge(x2, sdn, how='left', on=['IDTeams'])
    x2['ScoreDiffNorm'] = x2['ScoreDiffNorm'].fillna(0.)
    
    #Interactions
    inter = games[['IDTeam2','IDTeam1','Season','Pred']].rename(columns={'IDTeam2':'Target','IDTeam1':'Common'})
    inter['Pred'] = inter['Pred'] * -1
    inter = pd.concat((inter,games[['IDTeam1','IDTeam2','Season','Pred']].rename(columns={'IDTeam1':'Target','IDTeam2':'Common'})), axis=0, ignore_index=True).reset_index(drop=True)
    inter = inter[((inter['Season']<=int(season)) & (inter['Season']>int(season)-2))] #Only two years back and current regular season
    inter = pd.merge(inter, inter, how='inner', on=['Common','Season'])
    inter = inter[inter['Target_x'] != inter['Target_y']]
    #inter['ID'] = inter.apply(lambda r: '_'.join(map(str, [r['Season']+1, r['Target_x'].split('_')[1],r['Target_y'].split('_')[1]])), axis=1)
    inter['IDTeams'] = inter.apply(lambda r: '_'.join(map(str, [r['Target_x'].split('_')[1],r['Target_y'].split('_')[1]])), axis=1)
    inter = inter[['IDTeams','Pred_x']]
    inter = inter.groupby(['IDTeams'], as_index=False)[['Pred_x']].sum()
    inter = {k:int(v) for k, v in inter.values}
    
    x1['Inter'] = x1['IDTeams'].map(inter).fillna(0)
    x2['Inter'] = x2['IDTeams'].map(inter).fillna(0)
    test['Inter'] = test['IDTeams'].map(inter).fillna(0)
    col = [c for c in x1.columns if c not in ['ID', 'Team1','Team2', 'IDTeams','IDTeam1','IDTeam2','Pred','DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'NumOT', 'ScoreDiff']]
    
    reg = linear_model.HuberRegressor()
    reg.fit(x1[col], x1['Pred'])
    pred = reg.predict(x2[col]).clip(0.05, 0.95)
    print(x2.shape)
    print('Log Loss:', metrics.log_loss(x2['Pred'], pred))
    test['Pred'] = reg.predict(test[col])

    results.append(test)

2014
(67, 22)
Log Loss: 0.685003771728
2015
(67, 22)
Log Loss: 0.673741166775
2016
(67, 22)
Log Loss: 0.770371595411
2017
(67, 22)
Log Loss: 0.739854954286


In [2]:
results = pd.concat(results, axis=0, ignore_index=True).reset_index(drop=True)
#Testing for Sequence of Scoring
results = {k:float(v) for k,v in results[['ID','Pred']].values}
sub['Pred'] = sub['ID'].map(results).clip(0.05, 0.95).fillna(0.49)
sub[['ID','Pred']].to_csv('rh3p_submission.csv', index=False)

NameError: name 'pd' is not defined

In [17]:
x2
"""
['Season',
 'WLoc',
 'SecondaryTourney',
 'Team1Seed',
 'Team2Seed',
 'ScoreDiffNorm',
 'SeedDiff',
 'Inter']
"""


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney,ID,...,Team2,IDTeam1,IDTeam2,Team1Seed,Team2Seed,ScoreDiff,Pred,ScoreDiffNorm,SeedDiff,Inter
2050,2017,134,1243,95,1448,88,3,0,6,2017_1243_1448,...,1448,2017_1243,2017_1448,11.0,11.0,7,1.0,7,0.0,0.0
2051,2017,134,1291,67,1309,66,3,0,6,2017_1291_1309,...,1309,2017_1291,2017_1309,16.0,16.0,1,1.0,1,0.0,0.0
2052,2017,135,1413,67,1300,63,3,0,6,2017_1300_1413,...,1413,2017_1300,2017_1413,16.0,16.0,4,0.0,-4,0.0,0.0
2053,2017,135,1425,75,1344,71,3,0,6,2017_1344_1425,...,1425,2017_1344,2017_1425,11.0,11.0,4,0.0,-4,0.0,0.0
2054,2017,136,1112,100,1315,82,3,0,6,2017_1112_1315,...,1315,2017_1112,2017_1315,2.0,15.0,18,1.0,18,-13.0,5.0
2055,2017,136,1139,76,1457,64,3,0,6,2017_1139_1457,...,1457,2017_1139,2017_1457,4.0,13.0,12,1.0,12,-9.0,0.0
2056,2017,136,1196,80,1190,65,3,0,6,2017_1190_1196,...,1196,2017_1190,2017_1196,13.0,4.0,15,0.0,-15,9.0,4.0
2057,2017,136,1199,86,1195,80,3,0,6,2017_1195_1199,...,1199,2017_1195,2017_1199,14.0,3.0,6,0.0,-6,11.0,0.0
2058,2017,136,1211,66,1355,46,3,0,6,2017_1211_1355,...,1355,2017_1211,2017_1355,1.0,16.0,20,1.0,20,-15.0,3.0
2059,2017,136,1235,84,1305,73,3,0,6,2017_1235_1305,...,1305,2017_1235,2017_1305,5.0,12.0,11,1.0,11,-7.0,0.0


In [27]:
test.loc[test.ID=="2017_1243_1448"]

,ID,Pred,WLoc,SecondaryTourney,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,Team2Seed,SeedDiff,ScoreDiffNorm,Inter
1096,2017_1243_1448,0.509348,3,6,2017,1243,1448,1243_1448,2017_1243,2017_1448,11,11,0,0.0,0.0


In [31]:
# remove  ScoreDiffNorm
# 説明変数を選択
col = [c for c in x1.columns if c not in [
    'ID', 
    'Team1',
    'Team2', 
    'IDTeams',
    'IDTeam1',
    'IDTeam2',
    'Pred',
    'DayNum', 
    'WTeamID', 
    'WScore', 
    'LTeamID', 
    'LScore', 
    'NumOT', 
    'ScoreDiff',
    'ScoreDiffNorm'
]]
col

['Season',
 'WLoc',
 'SecondaryTourney',
 'Team1Seed',
 'Team2Seed',
 'SeedDiff',
 'Inter']

In [32]:
results = []
for season in sub['Season'].unique(): # 年度ごとに処理
    print(season)
    # 前年度以前のNCAAのデータを抜き出す
    x1 = games[((games['Season']<int(season)) & (games['SecondaryTourney']==6))]
    # 同年度以前のNCAA以外のデータを抜き出し追加
    x1 = pd.concat((x1,games[((games['Season']<int(int(season)+1)) & (games['SecondaryTourney']!=6))]), axis=0, 
                   ignore_index=True)
    # 同年度のNCAAのデータを抜き出す
    x2 = games[((games['Season']==int(season)) & (games['SecondaryTourney']==6))]
    test = sub[sub['Season']==season]

    sdn = x1.groupby(['IDTeams'], as_index=False)[['ScoreDiffNorm']].mean() # 同じ対戦カードの過去の点差の平均
    test = pd.merge(test, sdn, how='left', on=['IDTeams'])
    test['ScoreDiffNorm'] = test['ScoreDiffNorm'].fillna(0.)
    
    x2 = x2.drop("ScoreDiffNorm", axis=1)
    x2 = pd.merge(x2, sdn, how='left', on=['IDTeams'])
    x2['ScoreDiffNorm'] = x2['ScoreDiffNorm'].fillna(0.)
    
    #Interactions
    inter = games[['IDTeam2','IDTeam1','Season','Pred']].rename(columns={'IDTeam2':'Target','IDTeam1':'Common'})
    inter['Pred'] = inter['Pred'] * -1
    inter = pd.concat((inter,games[['IDTeam1','IDTeam2','Season','Pred']].rename(columns={'IDTeam1':'Target','IDTeam2':'Common'})), axis=0, ignore_index=True).reset_index(drop=True)
    inter = inter[((inter['Season']<=int(season)) & (inter['Season']>int(season)-2))] #Only two years back and current regular season
    inter = pd.merge(inter, inter, how='inner', on=['Common','Season'])
    inter = inter[inter['Target_x'] != inter['Target_y']]
    #inter['ID'] = inter.apply(lambda r: '_'.join(map(str, [r['Season']+1, r['Target_x'].split('_')[1],r['Target_y'].split('_')[1]])), axis=1)
    inter['IDTeams'] = inter.apply(lambda r: '_'.join(map(str, [r['Target_x'].split('_')[1],r['Target_y'].split('_')[1]])), axis=1)
    inter = inter[['IDTeams','Pred_x']]
    inter = inter.groupby(['IDTeams'], as_index=False)[['Pred_x']].sum()
    inter = {k:int(v) for k, v in inter.values}
    
    x1['Inter'] = x1['IDTeams'].map(inter).fillna(0)
    x2['Inter'] = x2['IDTeams'].map(inter).fillna(0)
    test['Inter'] = test['IDTeams'].map(inter).fillna(0)
    col = [c for c in x1.columns if c not in ['ID', 'Team1','Team2', 'IDTeams','IDTeam1','IDTeam2','Pred','DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'NumOT', 'ScoreDiff']]
    
    reg = linear_model.HuberRegressor()
    reg.fit(x1[col], x1['Pred'])
    pred = reg.predict(x2[col]).clip(0.05, 0.95)
    print(x2.shape)
    print('Log Loss:', metrics.log_loss(x2['Pred'], pred))
    test['Pred'] = reg.predict(test[col])

    results.append(test)

2014
(67, 22)
Log Loss: 0.685003771728
2015
(67, 22)
Log Loss: 0.673741166775
2016
(67, 22)
Log Loss: 0.770371595411
2017
(67, 22)
Log Loss: 0.739854954286


In [38]:
col = [c for c in x1.columns if c not in [
    'ID', 
    'Team1',
    'Team2', 
    'IDTeams',
    'IDTeam1',
    'IDTeam2',
    'Pred',
    'DayNum', 
    'WTeamID', 
    'WScore', 
    'LTeamID', 
    'LScore', 
    'NumOT', 
    'ScoreDiff',
    'ScoreDiffNorm',
 'Team1Seed',
 'Team2Seed',
 'Inter',
]]
"""
['Season',
 'WLoc',
 'SecondaryTourney',
 'Team1Seed',
 'Team2Seed',
 'SeedDiff',
 'Inter']
"""
col

['Season', 'WLoc', 'SecondaryTourney', 'SeedDiff']

In [39]:
results = []
for season in sub['Season'].unique(): # 年度ごとに処理
    print(season)
    # 前年度以前のNCAAのデータを抜き出す
    x1 = games[((games['Season']<int(season)) & (games['SecondaryTourney']==6))]
    # 同年度以前のNCAA以外のデータを抜き出し追加
    x1 = pd.concat((x1,games[((games['Season']<int(int(season)+1)) & (games['SecondaryTourney']!=6))]), axis=0, 
                   ignore_index=True)
    # 同年度のNCAAのデータを抜き出す
    x2 = games[((games['Season']==int(season)) & (games['SecondaryTourney']==6))]
    test = sub[sub['Season']==season]

    sdn = x1.groupby(['IDTeams'], as_index=False)[['ScoreDiffNorm']].mean() # 同じ対戦カードの過去の点差の平均
    test = pd.merge(test, sdn, how='left', on=['IDTeams'])
    test['ScoreDiffNorm'] = test['ScoreDiffNorm'].fillna(0.)
    
    x2 = x2.drop("ScoreDiffNorm", axis=1)
    x2 = pd.merge(x2, sdn, how='left', on=['IDTeams'])
    x2['ScoreDiffNorm'] = x2['ScoreDiffNorm'].fillna(0.)
    
    #Interactions
    inter = games[['IDTeam2','IDTeam1','Season','Pred']].rename(columns={'IDTeam2':'Target','IDTeam1':'Common'})
    inter['Pred'] = inter['Pred'] * -1
    inter = pd.concat((inter,games[['IDTeam1','IDTeam2','Season','Pred']].rename(columns={'IDTeam1':'Target','IDTeam2':'Common'})), axis=0, ignore_index=True).reset_index(drop=True)
    inter = inter[((inter['Season']<=int(season)) & (inter['Season']>int(season)-2))] #Only two years back and current regular season
    inter = pd.merge(inter, inter, how='inner', on=['Common','Season'])
    inter = inter[inter['Target_x'] != inter['Target_y']]
    #inter['ID'] = inter.apply(lambda r: '_'.join(map(str, [r['Season']+1, r['Target_x'].split('_')[1],r['Target_y'].split('_')[1]])), axis=1)
    inter['IDTeams'] = inter.apply(lambda r: '_'.join(map(str, [r['Target_x'].split('_')[1],r['Target_y'].split('_')[1]])), axis=1)
    inter = inter[['IDTeams','Pred_x']]
    inter = inter.groupby(['IDTeams'], as_index=False)[['Pred_x']].sum()
    inter = {k:int(v) for k, v in inter.values}
    
    x1['Inter'] = x1['IDTeams'].map(inter).fillna(0)
    x2['Inter'] = x2['IDTeams'].map(inter).fillna(0)
    test['Inter'] = test['IDTeams'].map(inter).fillna(0)
  
    reg = linear_model.HuberRegressor()
    reg.fit(x1[col], x1['Pred'])
    pred = reg.predict(x2[col]).clip(0.05, 0.95)
    print(x2.shape)
    print('Log Loss:', metrics.log_loss(x2['Pred'], pred))
    test['Pred'] = reg.predict(test[col])

    results.append(test)

2014
(67, 22)
Log Loss: 0.91181533934
2015
(67, 22)
Log Loss: 0.951579721474
2016
(67, 22)
Log Loss: 0.923835977373
2017
(67, 22)
Log Loss: 0.959793202138


In [40]:
col

['Season', 'WLoc', 'SecondaryTourney', 'SeedDiff']

In [41]:
results = []
for season in sub['Season'].unique(): # 年度ごとに処理
    print(season)
    # 前年度以前のNCAAのデータを抜き出す
    x1 = games[((games['Season']<int(season)) & (games['SecondaryTourney']==6))]
    # 同年度以前のNCAA以外のデータを抜き出し追加
#     x1 = pd.concat((x1,games[((games['Season']<int(int(season)+1)) & (games['SecondaryTourney']!=6))]), axis=0, 
#                    ignore_index=True)
    # 同年度のNCAAのデータを抜き出す
    x2 = games[((games['Season']==int(season)) & (games['SecondaryTourney']==6))]
    test = sub[sub['Season']==season]

    sdn = x1.groupby(['IDTeams'], as_index=False)[['ScoreDiffNorm']].mean() # 同じ対戦カードの過去の点差の平均
    test = pd.merge(test, sdn, how='left', on=['IDTeams'])
    test['ScoreDiffNorm'] = test['ScoreDiffNorm'].fillna(0.)
    
    x2 = x2.drop("ScoreDiffNorm", axis=1)
    x2 = pd.merge(x2, sdn, how='left', on=['IDTeams'])
    x2['ScoreDiffNorm'] = x2['ScoreDiffNorm'].fillna(0.)
    
    #Interactions
    inter = games[['IDTeam2','IDTeam1','Season','Pred']].rename(columns={'IDTeam2':'Target','IDTeam1':'Common'})
    inter['Pred'] = inter['Pred'] * -1
    inter = pd.concat((inter,games[['IDTeam1','IDTeam2','Season','Pred']].rename(columns={'IDTeam1':'Target','IDTeam2':'Common'})), axis=0, ignore_index=True).reset_index(drop=True)
    inter = inter[((inter['Season']<=int(season)) & (inter['Season']>int(season)-2))] #Only two years back and current regular season
    inter = pd.merge(inter, inter, how='inner', on=['Common','Season'])
    inter = inter[inter['Target_x'] != inter['Target_y']]
    #inter['ID'] = inter.apply(lambda r: '_'.join(map(str, [r['Season']+1, r['Target_x'].split('_')[1],r['Target_y'].split('_')[1]])), axis=1)
    inter['IDTeams'] = inter.apply(lambda r: '_'.join(map(str, [r['Target_x'].split('_')[1],r['Target_y'].split('_')[1]])), axis=1)
    inter = inter[['IDTeams','Pred_x']]
    inter = inter.groupby(['IDTeams'], as_index=False)[['Pred_x']].sum()
    inter = {k:int(v) for k, v in inter.values}
    
    x1['Inter'] = x1['IDTeams'].map(inter).fillna(0)
    x2['Inter'] = x2['IDTeams'].map(inter).fillna(0)
    test['Inter'] = test['IDTeams'].map(inter).fillna(0)
 
    reg = linear_model.HuberRegressor()
    reg.fit(x1[col], x1['Pred'])
    pred = reg.predict(x2[col]).clip(0.05, 0.95)
    print(x2.shape)
    print('Log Loss:', metrics.log_loss(x2['Pred'], pred))
    test['Pred'] = reg.predict(test[col])

    results.append(test)

2014


/home/akiyama/anaconda3/envs/keras2/lib/python3.5/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(67, 22)
Log Loss: 0.626529762936
2015
(67, 22)
Log Loss: 0.546160608281
2016
(67, 22)
Log Loss: 0.611810670846
2017
(67, 22)
Log Loss: 0.534516699184


In [42]:
col=['SeedDiff', 'Inter']
results = []
for season in sub['Season'].unique(): # 年度ごとに処理
    print(season)
    # 前年度以前のNCAAのデータを抜き出す
    x1 = games[((games['Season']<int(season)) & (games['SecondaryTourney']==6))]
    # 同年度以前のNCAA以外のデータを抜き出し追加
#     x1 = pd.concat((x1,games[((games['Season']<int(int(season)+1)) & (games['SecondaryTourney']!=6))]), axis=0, 
#                    ignore_index=True)
    # 同年度のNCAAのデータを抜き出す
    x2 = games[((games['Season']==int(season)) & (games['SecondaryTourney']==6))]
    test = sub[sub['Season']==season]

    sdn = x1.groupby(['IDTeams'], as_index=False)[['ScoreDiffNorm']].mean() # 同じ対戦カードの過去の点差の平均
    test = pd.merge(test, sdn, how='left', on=['IDTeams'])
    test['ScoreDiffNorm'] = test['ScoreDiffNorm'].fillna(0.)
    

    
    
    #Interactions
    inter = games[['IDTeam2','IDTeam1','Season','Pred']].rename(columns={'IDTeam2':'Target','IDTeam1':'Common'})
    inter['Pred'] = inter['Pred'] * -1
    inter = pd.concat((inter,games[['IDTeam1','IDTeam2','Season','Pred']].rename(columns={'IDTeam1':'Target','IDTeam2':'Common'})), axis=0, ignore_index=True).reset_index(drop=True)
    inter = inter[((inter['Season']<=int(season)) & (inter['Season']>int(season)-2))] #Only two years back and current regular season
    inter = pd.merge(inter, inter, how='inner', on=['Common','Season'])
    inter = inter[inter['Target_x'] != inter['Target_y']]
    #inter['ID'] = inter.apply(lambda r: '_'.join(map(str, [r['Season']+1, r['Target_x'].split('_')[1],r['Target_y'].split('_')[1]])), axis=1)
    inter['IDTeams'] = inter.apply(lambda r: '_'.join(map(str, [r['Target_x'].split('_')[1],r['Target_y'].split('_')[1]])), axis=1)
    inter = inter[['IDTeams','Pred_x']]
    inter = inter.groupby(['IDTeams'], as_index=False)[['Pred_x']].sum()
    inter = {k:int(v) for k, v in inter.values}
    
    x1['Inter'] = x1['IDTeams'].map(inter).fillna(0)
    x2['Inter'] = x2['IDTeams'].map(inter).fillna(0)
    test['Inter'] = test['IDTeams'].map(inter).fillna(0)
    col = [c for c in x1.columns if c not in ['ID', 'Team1','Team2', 'IDTeams','IDTeam1','IDTeam2','Pred','DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'NumOT', 'ScoreDiff']]
    
    reg = linear_model.HuberRegressor()
    reg.fit(x1[col], x1['Pred'])
    pred = reg.predict(x2[col]).clip(0.05, 0.95)
    print(x2.shape)
    print('Log Loss:', metrics.log_loss(x2['Pred'], pred))
    test['Pred'] = reg.predict(test[col])

    results.append(test)

2014


/home/akiyama/anaconda3/envs/keras2/lib/python3.5/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(67, 22)
Log Loss: 0.779695885355
2015
(67, 22)
Log Loss: 0.657213496905
2016
(67, 22)
Log Loss: 0.788645389221
2017
(67, 22)
Log Loss: 0.702825505758
